In [2]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np

import random
def cleanString(review):
    stopWords = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    returnString = ""
    sentence_token = tokenize.word_tokenize(review)
    idx_list = []
    for j in range(len(sentence_token)):
        #single_sentence = tokenize.word_tokenize(sentence_token[j])
        single_sentence=[lemmatizer.lemmatize(t) for t in sentence_token]
        single_sentence=[word for word in single_sentence if word.lower() not in stopWords]
        sentences_filtered = [(idx,lemmatizer.lemmatize(w.lower())) for idx,w in enumerate(single_sentence) 
                              if w.lower() not in stopWords and w.isalnum()]
        idx_list.append([x[0] for x in sentences_filtered])
        word_list = [x[1] for x in sentences_filtered]
        returnString = returnString + ' '.join(word_list) + ' '
    
    return returnString




def predict_class(sentence,classes):

    res,prob = predict_the_message_prob(sentence)

    return_list = []
    list1, list2 = (list(t) for t in zip(*sorted(zip(prob, res))))
    for l,t in zip(list1, list2):
        return_list.append({"intent": classes[t[-1]], "probability": str(l[-1])})
    print(return_list)
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    print(tag)
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = i['responses']
            break
    return result

def chatbot_response1(msg):
    words=[]
    classes = []
    documents = []
    ignore_words = ['?', '!']
    data_file = open('out1.json').read()
    intents = json.loads(data_file)
    for intent in list(intents['intents']):
        #print(intent['patterns'])
        w = nltk.word_tokenize(intent['patterns'])
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
    #words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
    #words = sorted(list(set(words)))

    #classes = sorted(list(set(classes)))

    #print (len(documents), "documents")

    #print (len(classes), "classes", classes)

    #print (len(words), "unique lemmatized words", words)

    ints = predict_class(msg,classes)
    res = getResponse(ints, intents)
    return res

def listToString(s):  
    
    # initialize an empty string 
    str1 = ""  
    
    # traverse in the string   
    for ele in s:  
        str1 += ele   
    
    # return string   
    return str1  

english_stemmer = nltk.stem.SnowballStemmer('english')
class StemmedTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        analyzer = super(TfidfVectorizer,self).build_analyzer()
        return lambda doc:(english_stemmer.stem(word) for word in analyzer(doc))
    
    
def cleanData(string1):
    articles = []
    n = 1
    for i in range(n):
        temp_string = cleanString(string1)
        articles.append(temp_string)
        print(str(i+1)+' of '+str(n)+" articles cleaned.",end='\r')
    
    return(articles)

def predict_the_message_prob(clntxt):
    print(text)
    clntxt=cleanData(text)
    word_model = pickle.load( open( "Python_word_preprocessing.sav", "rb" ) )
    char_model = pickle.load( open( "Pyhton_char_preprocessing.sav", "rb" ) )
    svd_model = pickle.load( open( "Pyhton_SVD_preprocessing.sav", "rb" ) )
    all_features = pickle.load( open( "train_features1_preprocessing.sav", "rb" ) )
    X_test1 = word_model.transform(clntxt)
    X_test2 = char_model.transform(clntxt)
    test_stack = hstack([X_test2,X_test1])
    test_stack=svd_model.transform(test_stack)
    model = pickle.load( open( "model.sav", "rb" ) )
    best_thread = pairwise_distances_argmin(
            X=test_stack,
            Y=all_features,
            metric='cosine'
        )
    print(best_thread)
    y_pred_class1 = model.predict_proba(all_features[best_thread])[0]
    y_pred_class2 = model.predict_proba(test_stack)
    #print(np.sort(y_pred_class2, axis=1)[:,-8:])
    #y_pred_class2 = model.predict_proba(test_stack)
    best_n = np.argsort(y_pred_class2, axis=1)[:,-8:]
    #best_n = np.argsort(probs, axis=1)[:,-n:]
    #print(y_pred_class1,y_pred_class2)
    return(best_n,np.sort(y_pred_class2, axis=1)[:,-8:])

def predict_the_message(text):
    print(text)
    clntxt=cleanData(text)
    word_model = pickle.load( open( "Python_word_preprocessing.sav", "rb" ) )
    char_model = pickle.load( open( "Pyhton_char_preprocessing.sav", "rb" ) )
    svd_model = pickle.load( open( "Pyhton_SVD_preprocessing.sav", "rb" ) )
    all_features = pickle.load( open( "train_features1_preprocessing.sav", "rb" ) )
    X_test1 = word_model.transform(clntxt)
    X_test2 = char_model.transform(clntxt)
    test_stack = hstack([X_test2,X_test1])
    test_stack=svd_model.transform(test_stack)
    model = pickle.load( open( "model.sav", "rb" ) )
    best_thread = pairwise_distances_argmin(
            X=test_stack,
            Y=all_features,
            metric='cosine'
        )
    print(best_thread)
    y_pred_class1 = model.predict(all_features[best_thread])
    y_pred_class2 = model.predict(test_stack)
    print(y_pred_class1,y_pred_class2)
    return(y_pred_class1,y_pred_class2)

def clicked (value):
    clearscreen()   ## to clear the screen
    label1 = Label(root, text=value, font=("Verdana", 12) )
    label1.pack()

    #Create Chat window
    ChatLog = Text(root, bd=0, bg="white", height="8", width="50", font="Arial")
    
    ChatLog.insert(END, "AASKK:  Hello!!, Please type your question " '\n\n')
    ChatLog.config(foreground="#442265", font=("Verdana", 12 ),borderwidth = 5)

    ChatLog.config(state=DISABLED)

    #Bind scrollbar to Chat window
    scrollbar = Scrollbar(root, command=ChatLog.yview, cursor="heart")
    ChatLog['yscrollcommand'] = scrollbar.set
    
    #Create the box to enter message
    EntryBox = Text(root, bd=0, bg="white",width="29", height="5", font="Arial",borderwidth = 5)
    #EntryBox.bind("<Return>", send)

    #Create Button to send message
    SendButton = Button(root, font=("Verdana",12,'bold'), text="Send", width="11", height=4,
                        bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',borderwidth = 5,
                        command= lambda: send(EntryBox,ChatLog,value) )

    #Place all components on the screen
    scrollbar.place(x=480,y=25, height=375)
    ChatLog.place(x=6,y=25, height=375, width=480)
    EntryBox.place(x=140, y=403, height=65, width=360)
    SendButton.place(x=6, y=403, height=65, width=135)



def clearscreen():
    list = root.pack_slaves()
    for l in list:
        l.destroy()    
        
def send(EntryBox,ChatLog,value):
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    
    if msg == '':
        messagebox.showinfo("Allert", "Please enter your question and then click send")
        
    elif msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg,value)
        ChatLog.insert(END, "AASKK: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
        

def chatbot_response(msg,value):
    
   # msg =cleanData(msg)  ##output is articals
    
    if value == "Python":
        output,output2= predict_the_message(msg)
        
        label=  pickle.load( open( "labelEncoder.pickle", "rb" ) )
        className=label.classes_[output]
        className1=label.classes_[output2]
        print(label.classes_[output])
        print(label.classes_[output2])
        str1 = ''.join(className1)
        dict_label=  pickle.load( open( "LabelDictionary.pickle", "rb" ) )
        #dict_label[listToString(['Data Analysis'])]        
        HardCodedResult1=dict_label[listToString(label.classes_[output])]
        HardCodedResult2=dict_label[listToString(label.classes_[output2])]
        match_object = re.search(r'CATEGORY(.*)', str1, flags=re.IGNORECASE)
        if match_object:
  #      if re.search(r"CAT(.*)", str1, flags=re.IGNORECASE):
            index=match_object.group(1)
            answers=pickle.load( open( "data_df.pickle", "rb" ) )
            #answers=  pickle.load( open( "answers.sav", "rb" ) )
            className=answers.iloc[int(index)]
           # className=tlist[float(match_object.group(1))-1]
            print(className)
        #if re.match(CAT\d+, output) is not None:
 
        return(str(output)+str(className1)+"\n"+str(HardCodedResult1))  
    elif value == "Statistics":
        output = predict_the_message(msg)
        
        label=  pickle.load( open( "labelEncoder.pickle", "rb" ) )
        className=label.classes_[output]
        className1=label.classes_[output2]
        print(label.classes_[output])
        print(label.classes_[output2])
        str1 = ''.join(className1)
        dict_label=  pickle.load( open( "LabelDictionary.pickle", "rb" ) )
        #dict_label[listToString(['Data Analysis'])]        
        HardCodedResult1=dict_label[listToString(label.classes_[output])]
        HardCodedResult2=dict_label[listToString(label.classes_[output2])]
        match_object = re.search(r'CAT(.*)', str1, flags=re.IGNORECASE)
        if match_object:
  #      if re.search(r"CAT(.*)", str1, flags=re.IGNORECASE):
            index=match_object.group(1)
            answers=pickle.load( open( "data_df.pickle", "rb" ) )
            #answers=  pickle.load( open( "answers.sav", "rb" ) )
            className=answers.iloc[int(index)]
           # className=tlist[float(match_object.group(1))-1]
            print(className)
        #if re.match(CAT\d+, output) is not None:
     
        import wikipediaapi
        import wikipedia
        wiki_wiki = wikipediaapi.Wikipedia('en')
        print(className1)
        wikiResults = wikipedia.search(className)
        result = wikiResults[0]
        page_py = wiki_wiki.page(result)
        print(className)
        print(page_py.fullurl)
        return(str(output)+str(className1)+"\n"+page_py.fullurl+"\n"+HardCodedResult1+"\n"+HardCodedResult2)  
    elif value == "Supervised Learning":
        output = predict_the_message(msg)
        
        label=  pickle.load( open( "labelEncoder.pickle", "rb" ) )
        className=label.classes_[output]
        className1=label.classes_[output2]
        print(label.classes_[output])
        print(label.classes_[output2])
        str1 = ''.join(className1)
        dict_label=  pickle.load( open( "LabelDictionary.pickle", "rb" ) )
        #dict_label[listToString(['Data Analysis'])]        
        HardCodedResult1=dict_label[listToString(label.classes_[output])]
        HardCodedResult2=dict_label[listToString(label.classes_[output2])]
        match_object = re.search(r'CAT(.*)', str1, flags=re.IGNORECASE)
        if match_object:
  #      if re.search(r"CAT(.*)", str1, flags=re.IGNORECASE):
            index=match_object.group(1)
            answers=pickle.load( open( "data_df.pickle", "rb" ) )
            #answers=  pickle.load( open( "answers.sav", "rb" ) )
            className=answers.iloc[int(index)]
           # className=tlist[float(match_object.group(1))-1]
            print(className)
        #if re.match(CAT\d+, output) is not None:
     
        import wikipediaapi
        import wikipedia
        wiki_wiki = wikipediaapi.Wikipedia('en')
        print(className1)
        wikiResults = wikipedia.search(className)
        result = wikiResults[0]
        page_py = wiki_wiki.page(result)
        print(className)
        print(page_py.fullurl)
        return(str(output)+str(className1)+"\n"+page_py.fullurl+"\n"+HardCodedResult1+"\n"+HardCodedResult2)  
    elif value == "Ensemble Techniques":
        output = predict_the_message(msg)
        
        label=  pickle.load( open( "labelEncoder.pickle", "rb" ) )
        className=label.classes_[output]
        className1=label.classes_[output2]
        print(label.classes_[output])
        print(label.classes_[output2])
        str1 = ''.join(className1)
        dict_label=  pickle.load( open( "LabelDictionary.pickle", "rb" ) )
        #dict_label[listToString(['Data Analysis'])]        
        HardCodedResult1=dict_label[output]
        HardCodedResult2=dict_label[output2]
        match_object = re.search(r'CAT(.*)', str1, flags=re.IGNORECASE)
        if match_object:
  #      if re.search(r"CAT(.*)", str1, flags=re.IGNORECASE):
            index=match_object.group(1)
            answers=pickle.load( open( "data_df.pickle", "rb" ) )
            #answers=  pickle.load( open( "answers.sav", "rb" ) )
            className=answers.iloc[int(index)]
           # className=tlist[float(match_object.group(1))-1]
            print(className)
        #if re.match(CAT\d+, output) is not None:
     
        import wikipediaapi
        import wikipedia
        wiki_wiki = wikipediaapi.Wikipedia('en')
        print(className1)
        wikiResults = wikipedia.search(className)
        result = wikiResults[0]
        page_py = wiki_wiki.page(result)
        print(className)
        print(page_py.fullurl)
        return(str(output)+str(className1)+"\n"+page_py.fullurl+"\n"+HardCodedResult1+"\n"+HardCodedResult2)  
    elif value == "Unsupervised learning":
        output = predict_the_message(msg)
        
        label=  pickle.load( open( "labelEncoder.pickle", "rb" ) )
        className=label.classes_[output]
        className1=label.classes_[output2]
        print(label.classes_[output])
        print(label.classes_[output2])
        str1 = ''.join(className1)
        dict_label=  pickle.load( open( "LabelDictionary.pickle", "rb" ) )
        #dict_label[listToString(['Data Analysis'])]        
        HardCodedResult1=dict_label[listToString(label.classes_[output])]
        HardCodedResult2=dict_label[listToString(label.classes_[output2])]
        match_object = re.search(r'CAT(.*)', str1, flags=re.IGNORECASE)
        if match_object:
  #      if re.search(r"CAT(.*)", str1, flags=re.IGNORECASE):
            index=match_object.group(1)
            #answers=pickle.load( open( "data_df.pickle", "rb" ) )
            answers=  pickle.load( open( "answers.sav", "rb" ) )
            className=answers.iloc[int(index)]
           # className=tlist[float(match_object.group(1))-1]
            print(className)
        #if re.match(CAT\d+, output) is not None:
     
        import wikipediaapi
        import wikipedia
        wiki_wiki = wikipediaapi.Wikipedia('en')
        print(className1)
        wikiResults = wikipedia.search(className)
        result = wikiResults[0]
        page_py = wiki_wiki.page(result)
        print(className)
        print(page_py.fullurl)
        return(str(output)+str(className1)+"\n"+page_py.fullurl+"\n"+HardCodedResult1+"\n"+HardCodedResult2)    
    else:
        Result=chatbot_response1(msg) 
        return(Result)
#Description: This is a chat bot GUI


#Import the library

from tkinter import *
from PIL import ImageTk,Image
from tkinter import messagebox

root = Tk()
#Name of the chat Bot
root.title("AASKK") 
#Geometry of the chat Bot
root.geometry("500x600")
root.resizable(width=FALSE, height=FALSE)


# Creating Background Image
canvas=Canvas(root,width=500,height=600)
#give this image path. image should be in png format.
#image=ImageTk.PhotoImage(Image.open("wall_e.png"))
#canvas.create_image(0,0,anchor=NW,image=image)
canvas.place(height=600, x=0, y=0)

def donothing():
   filewin = Toplevel(root)
   button = Button(filewin, text="Do nothing button")
   button.pack()
   
menubar = Menu(root)
filemenu = Menu(menubar, tearoff=0)
filemenu.add_command(label="New", command=donothing)
filemenu.add_command(label="Open", command=donothing)
filemenu.add_command(label="Save", command=donothing)
filemenu.add_command(label="Save as...", command=donothing)
filemenu.add_command(label="Close", command=donothing)

filemenu.add_separator()

filemenu.add_command(label="Exit", command=root.quit)
menubar.add_cascade(label="File", menu=filemenu)
editmenu = Menu(menubar, tearoff=0)
editmenu.add_command(label="Undo", command=donothing)

editmenu.add_separator()

editmenu.add_command(label="Cut", command=donothing)
editmenu.add_command(label="Copy", command=donothing)
editmenu.add_command(label="Paste", command=donothing)
editmenu.add_command(label="Delete", command=donothing)
editmenu.add_command(label="Select All", command=donothing)

menubar.add_cascade(label="Edit", menu=editmenu)
helpmenu = Menu(menubar, tearoff=0)
helpmenu.add_command(label="Help Index", command=donothing)
helpmenu.add_command(label="About...", command=donothing)
menubar.add_cascade(label="Help", menu=helpmenu)

root.config(menu=menubar)

label2 = Label(root, text="Welcome", font=("Arial", 18) )
label2.pack(padx= 20, pady =10)

label2 = Label(root, text="Please select the topic from which you want to learn", font=("Arial", 12) )
label2.pack(padx= 20, pady =20)


TOPICS = [("Python","Python"),
          ("Statistics","Statistics"),
          ("Supervised Learning","Supervised Learning"),
          ("Ensemble Techniques","Ensemble Techniques"),
          ("Unsupervised learning","Unsupervised learning"),
          ("Not Sure about the topic","All Topic")
         ]
option = StringVar()  
option.set("Python")
  
for text,topics in TOPICS:
    r =Radiobutton(root, text = text, variable=option , value=topics, font=("Arial", 12))
    r.pack(fill=Y,anchor = W)
Button2 = Button(root, text="Submit",fg='#ffffff',
                 bg="#32de97",activebackground="#3c9d9b",font=("Arial", 12),
                 command = lambda : clicked(option.get()))

Button2.pack(padx= 20, pady =20)
from sklearn.metrics.pairwise import pairwise_distances_argmin_min
from sklearn.metrics.pairwise import pairwise_distances_argmin
root.mainloop()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KarthikeyanNatarajan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\KarthikeyanNatarajan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


what is AI, Machine Learning, Big Data computing and big data?
[86] 1 articles cleaned.
[45] [45]
['Introduction']
['Introduction']
Does python has features of data analytics
[478]1 articles cleaned.
[62] [62]
['Python Introduction']
['Python Introduction']
Hello
[147]1 articles cleaned.
[41] [41]
['Greetings']
['Greetings']
Linear Regression formula
[256]1 articles cleaned.
[78] [78]
['Types of Algorithm']
['Types of Algorithm']
unable to understand knn
[206]1 articles cleaned.
[55] [55]
['Nurel Network']
['Nurel Network']
Which library is used for machine learning models
[216]1 articles cleaned.
[61] [45]
['Probability - Meaning and concepts']
['Introduction']
did not get neural networks
[62] 1 articles cleaned.
[55] [55]
['Nurel Network']
['Nurel Network']


In [1]:
import csv
import numpy as np
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk.stem
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet, stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import WordNetLemmatizer
from scipy.sparse import hstack
# Import necessary libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import math
import string
import re
from sklearn.preprocessing import LabelEncoder
import os
import pickle
import sys
from IPython.display import display, HTML
import urllib
import gzip
import nltk
from nltk import tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KarthikeyanNatarajan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KarthikeyanNatarajan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\KarthikeyanNatarajan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
dict_label=  pickle.load( open( "LabelDictionary.pickle", "rb" ) )


In [4]:
dict_label

{'Python Introduction': 'https://beginnersbook.com/2018/01/introduction-to-python-programming/ ',
 'Lists': 'https://chrisalbon.com/#python',
 'Dictionary': 'https://chrisalbon.com/#python',
 'File': 'https://chrisalbon.com/#python',
 'Numpy': 'https://en.wikipedia.org/wiki/NumPy',
 'Ploting': 'https://en.wikipedia.org/wiki/Matplotlib',
 'Pandas': 'https://nbviewer.jupyter.org/github/fonnesbeck/Bios8366/tree/master/notebooks/',
 'PivotTable': 'https://nbviewer.jupyter.org/github/fonnesbeck/Bios8366/tree/master/notebooks/',
 'GroupBy': 'https://nbviewer.jupyter.org/github/fonnesbeck/Bios8366/tree/master/notebooks/',
 'Missing': 'https://www.kaggle.com/dark4user/python-programming-from-a-to-z ',
 'Misc': 'https://www.kaggle.com/dark4user/python-programming-from-a-to-z ',
 'String': 'https://chrisalbon.com/#python',
 'Statistics': 'https://chrisalbon.com/#statistics',
 'Sets': 'https://chrisalbon.com/#python',
 'RegularExpression': 'https://chrisalbon.com/#regex',
 'Data Analysis': 'https

In [ ]:
intents

In [ ]:
classes

In [ ]:
>>> list1 = [3,2,4,1, 1]
>>> list2 = ['three', 'two', 'four', 'one', 'one2']
>>> list1, list2 = zip(*sorted(zip(list1, list2)))
>>> list1
(1, 1, 2, 3, 4)
>>> list2 
('one', 'one2', 'two', 'three', 'four')